In [ ]:
import networkx as nx
import math
import random

In [ ]:
def compute_stationary_distribution(G, teleportation=0.15, tol=1e-8, max_iter=100):
    """
    Computes the stationary distribution for a weighted directed graph G,
    using a PageRank-like power method with teleportation.
    
    The random walker with probability (1 - teleportation) follows
    out-links proportional to their weight, and with probability 
    teleportation jumps uniformly to any node.
    
    Parameters:
      G:             A NetworkX DiGraph with optional 'weight' on edges.
      teleportation: The teleportation probability (typically 0.15; i.e. d=0.85).
      tol:           Convergence tolerance.
      max_iter:      Maximum number of iterations.
      
    Returns:
      A dictionary mapping each node to its stationary probability.
    """
    # Initialize with a uniform distribution
    N = G.number_of_nodes()
    p = {node: 1.0 / N for node in G.nodes()}
    
    for _ in range(max_iter):
        # Start with the teleportation contribution for every node.
        new_p = {node: teleportation / N for node in G.nodes()}
        
        # For each node, distribute its probability mass along its outgoing edges.
        for node in G.nodes():
            # Get all outgoing edges from node along with their weight.
            out_edges = list(G.out_edges(node, data=True))
            if len(out_edges) == 0:
                # Final node: no outlinks. Distribute its (follow) probability uniformly.
                contribution = (1 - teleportation) * p[node] / N
                for dest in G.nodes():
                    new_p[dest] += contribution
            else:
                # Total weight of node's out-going links.
                total_weight = sum(data.get('weight', 1) for (_, _, data) in out_edges)
                # Distribute node's probability mass proportional to each link’s weight.
                for (_, neighbor, data) in out_edges:
                    weight = data.get('weight', 1)
                    new_p[neighbor] += (1 - teleportation) * p[node] * (weight / total_weight)
                    
        # Check convergence (L1 norm)
        err = sum(abs(new_p[node] - p[node]) for node in G.nodes())
        p = new_p
        if err < tol:
            break
    return p

def compute_map_equation(G, partition, p, teleportation=0.15):
    """
    Computes the map equation (description length L) for a given partition of the graph.
    
    The graph G is a weighted, directed graph (NetworkX DiGraph). The partition is
    a dictionary mapping nodes to module IDs. The stationary distribution p
    is a dictionary {node: probability} computed previously. 
    
    The exit probability for a node is computed from the (1 - teleportation) part of the dynamics,
    i.e., only flows that follow actual links (and leave the current module) are coded.
    
    Returns:
      L_total: The total description length (in bits) for the partition.
    """
    # Group nodes by their assigned module.
    modules = {}
    for node, module in partition.items():
        modules.setdefault(module, []).append(node)
    
    # Pre-compute the total out-weight of each node.
    out_weight = {}
    for node in G.nodes():
        out_edges = list(G.out_edges(node, data=True))
        if len(out_edges) == 0:
            out_weight[node] = 0
        else:
            total = sum(data.get('weight', 1) for (_, _, data) in out_edges)
            out_weight[node] = total
    
    # For each module, compute the total node probability and the exit probability.
    module_info = {}
    for module, nodes in modules.items():
        p_module = sum(p[node] for node in nodes)
        exit_module = 0.0
        # For each node in the module, add the probability that the random walker leaves the module.
        for node in nodes:
            if out_weight[node] > 0:
                # Check each outlink; if the neighbor lies outside the module then count this flow.
                for _, neighbor, data in G.out_edges(node, data=True):
                    if partition[neighbor] != module:
                        weight = data.get('weight', 1)
                        exit_module += p[node] * (1 - teleportation) * (weight / out_weight[node])
        module_info[module] = {'p_module': p_module, 'exit': exit_module, 'nodes': nodes}
    
    # Total exit probability (the inter-module flow) summed over all modules.
    q_exit = sum(info['exit'] for info in module_info.values())
    
    # Compute the entropy of the inter-module (exit) codebook.
    H_exit = 0.0
    if q_exit > 0:
        for info in module_info.values():
            if info['exit'] > 0:
                prob = info['exit'] / q_exit
                H_exit -= prob * math.log(prob, 2)
    
    # Compute the contribution to L from within each module.
    L_within = 0.0
    for module, info in module_info.items():
        p_module = info['p_module']
        exit_module = info['exit']
        total_flow = p_module + exit_module  # Total weight for the module's codebook.
        H_module = 0.0
        # First, include the exit code for the module.
        if exit_module > 0:
            prob_exit = exit_module / total_flow
            H_module -= prob_exit * math.log(prob_exit, 2)
        # Next, add the contribution from each node in the module.
        for node in info['nodes']:
            if p[node] > 0:
                prob_node = p[node] / total_flow
                H_module -= prob_node * math.log(prob_node, 2)
        # Multiply the module’s entropy by its total weight.
        L_within += total_flow * H_module
    
    # Combine the inter-module and within-module parts.
    L_total = q_exit * H_exit + L_within
    return L_total

def infomap_greedy(G, teleportation=0.15, max_iter=100):
    """
    Finds a (locally) optimal partition of graph G to minimize the map equation L
    using a greedy algorithm.
    
    The algorithm initializes with every node in its own module and then repeatedly 
    moves each node to a candidate module (among its neighbors' modules) if the move 
    reduces the total description length.
    
    Parameters:
      G:            A NetworkX DiGraph representing the network.
      teleportation: The teleportation probability for the random walker.
      max_iter:     Maximum iterations over all nodes.
      
    Returns:
      partition:  A dictionary mapping node to its module label.
      current_L:  The final (optimized) description length.
    """
    # Initialize partition: each node in its own module.
    partition = {node: node for node in G.nodes()}
    # Compute the stationary distribution for the random walk on G.
    p = compute_stationary_distribution(G, teleportation=teleportation)
    # Compute the initial description length.
    current_L = compute_map_equation(G, partition, p, teleportation=teleportation)
    
    improvement = True
    iter_count = 0
    # Repeat until no move improves L or maximum iterations is reached.
    while improvement and iter_count < max_iter:
        improvement = False
        iter_count += 1
        nodes = list(G.nodes())
        random.shuffle(nodes)  # Process nodes in random order to reduce bias.
        for node in nodes:
            current_module = partition[node]
            # Determine candidate modules:
            # (Use both out-going and in-coming neighbor modules.)
            candidate_modules = set()
            for (_, neighbor) in G.out_edges(node):
                candidate_modules.add(partition[neighbor])
            for (neighbor, _) in G.in_edges(node):
                candidate_modules.add(partition[neighbor])
            # Always include the current module.
            candidate_modules.add(current_module)
            
            best_module = current_module
            best_L = current_L
            # Test moving node to each candidate module.
            for module in candidate_modules:
                if module == current_module:
                    continue
                # Temporarily reassign node to the candidate module.
                old_module = partition[node]
                partition[node] = module
                new_L = compute_map_equation(G, partition, p, teleportation=teleportation)
                # If the move improves (i.e. lowers) L, record it.
                if new_L < best_L:
                    best_L = new_L
                    best_module = module
                # Revert the temporary move.
                partition[node] = old_module
            # If a better module was found, perform the move permanently.
            if best_module != current_module:
                partition[node] = best_module
                current_L = best_L
                improvement = True
        # (A more advanced implementation would also try merging modules and
        # refining using techniques like simulated annealing.)
    return partition, current_L

In [ ]:
G = nx.DiGraph()

# Add nodes (for clarity, nodes are labeled with integers)
for i in range(1, 8):
    G.add_node(i)

# Cluster 1: nodes 1,2,3 (strongly connected)
G.add_edge(1, 2, weight=3)
G.add_edge(2, 1, weight=3)
G.add_edge(2, 3, weight=3)
G.add_edge(3, 1, weight=3)

# Cluster 2: nodes 4,5,6,7 (strongly connected)
G.add_edge(4, 5, weight=3)
G.add_edge(5, 6, weight=3)
G.add_edge(6, 7, weight=3)
G.add_edge(7, 4, weight=3)

# Few links between clusters (weaker connections)
G.add_edge(3, 4, weight=1)
G.add_edge(7, 1, weight=1)

# Run the greedy Infomap-like community detection.
partition, L_value = infomap_greedy(G, teleportation=0.15, max_iter=100)
print("Optimized Partition (node -> module):")
for node in sorted(partition.keys()):
    print(f"  Node {node} -> Module {partition[node]}")
print("\nFinal map equation L (in bits per step):", L_value)

Optimized Partition (node -> module):
  Node 1 -> Module 2
  Node 2 -> Module 2
  Node 3 -> Module 2
  Node 4 -> Module 5
  Node 5 -> Module 5
  Node 6 -> Module 5
  Node 7 -> Module 5

Final map equation L (in bits per step): 2.1034831759741053
